In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler

In [3]:
online_news_raw_data = os.path.join(".", "Database Files", "OnlineNewsPopularity.csv")

In [4]:
online_news_df = pd.read_csv(online_news_raw_data)
online_news_df.head(10)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505
5,http://mashable.com/2013/01/07/beewi-smart-toys/,731.0,10.0,370.0,0.559889,1.0,0.698198,2.0,2.0,0.0,...,0.136364,0.6,-0.195000,-0.400,-0.100000,0.642857,0.214286,0.142857,0.214286,855
6,http://mashable.com/2013/01/07/bodymedia-armba...,731.0,8.0,960.0,0.418163,1.0,0.549834,21.0,20.0,20.0,...,0.100000,1.0,-0.224479,-0.500,-0.050000,0.000000,0.000000,0.500000,0.000000,556
7,http://mashable.com/2013/01/07/canon-poweshot-n/,731.0,12.0,989.0,0.433574,1.0,0.572108,20.0,20.0,20.0,...,0.100000,1.0,-0.242778,-0.500,-0.050000,1.000000,0.500000,0.500000,0.500000,891
8,http://mashable.com/2013/01/07/car-of-the-futu...,731.0,11.0,97.0,0.670103,1.0,0.836735,2.0,0.0,0.0,...,0.400000,0.8,-0.125000,-0.125,-0.125000,0.125000,0.000000,0.375000,0.000000,3600
9,http://mashable.com/2013/01/07/chuck-hagel-web...,731.0,10.0,231.0,0.636364,1.0,0.797101,4.0,1.0,1.0,...,0.100000,0.5,-0.238095,-0.500,-0.100000,0.000000,0.000000,0.500000,0.000000,710


In [5]:
# Clean data by only keeping columns we want to use
condensed_online_news_df = online_news_df.iloc[:,[2, 3, 48, 49, 50, 53, 57, 60]]
condensed_online_news_df.head(10)

,n_tokens_title,n_tokens_content,rate_positive_words,rate_negative_words,avg_positive_polarity,avg_negative_polarity,title_sentiment_polarity,shares
0,12.0,219.0,0.769231,0.230769,0.378636,-0.350000,-0.187500,593
1,9.0,255.0,0.733333,0.266667,0.286915,-0.118750,0.000000,711
2,9.0,211.0,0.857143,0.142857,0.495833,-0.466667,0.000000,1500
3,9.0,531.0,0.666667,0.333333,0.385965,-0.369697,0.000000,1200
4,13.0,1072.0,0.860215,0.139785,0.411127,-0.220192,0.136364,505
5,10.0,370.0,0.523810,0.476190,0.350610,-0.195000,0.214286,855
6,8.0,960.0,0.827957,0.172043,0.402039,-0.224479,0.000000,556
7,12.0,989.0,0.846939,0.153061,0.427720,-0.242778,0.500000,891
8,11.0,97.0,0.600000,0.400000,0.566667,-0.125000,0.000000,3600
9,10.0,231.0,0.562500,0.437500,0.298413,-0.238095,0.000000,710


In [6]:
# Bucket 'shares' column into categories so we can measure popularity
bins = [0, 500, 1000, 100000, 1000000]
labels = ["Less Shareable", "Somewhat Shareable", "Highly Shareable", "Extremely Shareable"]
condensed_online_news_df["Popularity"] = pd.cut(condensed_online_news_df[" shares"], bins, labels=labels)
condensed_online_news_df.head(10)

C:\Users\Michelle\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,n_tokens_title,n_tokens_content,rate_positive_words,rate_negative_words,avg_positive_polarity,avg_negative_polarity,title_sentiment_polarity,shares,Popularity
0,12.0,219.0,0.769231,0.230769,0.378636,-0.350000,-0.187500,593,Somewhat Shareable
1,9.0,255.0,0.733333,0.266667,0.286915,-0.118750,0.000000,711,Somewhat Shareable
2,9.0,211.0,0.857143,0.142857,0.495833,-0.466667,0.000000,1500,Highly Shareable
3,9.0,531.0,0.666667,0.333333,0.385965,-0.369697,0.000000,1200,Highly Shareable
4,13.0,1072.0,0.860215,0.139785,0.411127,-0.220192,0.136364,505,Somewhat Shareable
5,10.0,370.0,0.523810,0.476190,0.350610,-0.195000,0.214286,855,Somewhat Shareable
6,8.0,960.0,0.827957,0.172043,0.402039,-0.224479,0.000000,556,Somewhat Shareable
7,12.0,989.0,0.846939,0.153061,0.427720,-0.242778,0.500000,891,Somewhat Shareable
8,11.0,97.0,0.600000,0.400000,0.566667,-0.125000,0.000000,3600,Highly Shareable
9,10.0,231.0,0.562500,0.437500,0.298413,-0.238095,0.000000,710,Somewhat Shareable


In [8]:
del condensed_online_news_df[" shares"]
condensed_online_news_df.head(10)

,n_tokens_title,n_tokens_content,rate_positive_words,rate_negative_words,avg_positive_polarity,avg_negative_polarity,title_sentiment_polarity,Popularity
0,12.0,219.0,0.769231,0.230769,0.378636,-0.350000,-0.187500,Somewhat Shareable
1,9.0,255.0,0.733333,0.266667,0.286915,-0.118750,0.000000,Somewhat Shareable
2,9.0,211.0,0.857143,0.142857,0.495833,-0.466667,0.000000,Highly Shareable
3,9.0,531.0,0.666667,0.333333,0.385965,-0.369697,0.000000,Highly Shareable
4,13.0,1072.0,0.860215,0.139785,0.411127,-0.220192,0.136364,Somewhat Shareable
5,10.0,370.0,0.523810,0.476190,0.350610,-0.195000,0.214286,Somewhat Shareable
6,8.0,960.0,0.827957,0.172043,0.402039,-0.224479,0.000000,Somewhat Shareable
7,12.0,989.0,0.846939,0.153061,0.427720,-0.242778,0.500000,Somewhat Shareable
8,11.0,97.0,0.600000,0.400000,0.566667,-0.125000,0.000000,Highly Shareable
9,10.0,231.0,0.562500,0.437500,0.298413,-0.238095,0.000000,Somewhat Shareable


## Split the Data into Training and Testing Sets

In [9]:
# Create our features
X = condensed_online_news_df.drop("Popularity", axis=1)

X = pd.get_dummies(X)
# Create our target
y = condensed_online_news_df["Popularity"]
X

,n_tokens_title,n_tokens_content,rate_positive_words,rate_negative_words,avg_positive_polarity,avg_negative_polarity,title_sentiment_polarity
0,12.0,219.0,0.769231,0.230769,0.378636,-0.350000,-0.187500
1,9.0,255.0,0.733333,0.266667,0.286915,-0.118750,0.000000
2,9.0,211.0,0.857143,0.142857,0.495833,-0.466667,0.000000
3,9.0,531.0,0.666667,0.333333,0.385965,-0.369697,0.000000
4,13.0,1072.0,0.860215,0.139785,0.411127,-0.220192,0.136364
...,...,...,...,...,...,...,...
39639,11.0,346.0,0.722222,0.277778,0.333791,-0.260000,0.000000
39640,12.0,328.0,0.812500,0.187500,0.374825,-0.211111,1.000000
39641,10.0,442.0,0.576923,0.423077,0.307273,-0.356439,0.136364
39642,6.0,682.0,0.466667,0.533333,0.236851,-0.205246,0.000000


In [10]:
X.describe()

,n_tokens_title,n_tokens_content,rate_positive_words,rate_negative_words,avg_positive_polarity,avg_negative_polarity,title_sentiment_polarity
count,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000
mean,10.398749,546.514731,0.682150,0.287934,0.353825,-0.259524,0.071425
std,2.114037,471.107508,0.190206,0.156156,0.104542,0.127726,0.265450
min,2.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000
25%,9.000000,246.000000,0.600000,0.185185,0.306244,-0.328383,0.000000
50%,10.000000,409.000000,0.710526,0.280000,0.358755,-0.253333,0.000000
75%,12.000000,716.000000,0.800000,0.384615,0.411428,-0.186905,0.150000
max,23.000000,8474.000000,1.000000,1.000000,1.000000,0.000000,1.000000


In [11]:
# Check the balance of our target values
y.value_counts()

Highly Shareable       27162
Somewhat Shareable     11335
Less Shareable          1089
Extremely Shareable       58
Name: Popularity, dtype: int64

In [12]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X,
    y, random_state=1, stratify=y)

## Balanced Random Forest Classifer

In [13]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=128, random_state=1) 

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [14]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1", "Actual 2", "Actual 3"], columns=["Predicted 0", "Predicted 1", "Predicted 2", "Predicted 3"])

cm_df

,Predicted 0,Predicted 1,Predicted 2,Predicted 3
Actual 0,6,4,2,2
Actual 1,2240,1748,1626,1177
Actual 2,64,72,72,64
Actual 3,838,606,751,639


In [15]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, predictions)

0.2940382921914024

In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                           pre       rec       spe        f1       geo       iba       sup

Extremely Shareable       0.00      0.43      0.68      0.00      0.54      0.29        14
   Highly Shareable       0.72      0.26      0.78      0.38      0.45      0.19      6791
     Less Shareable       0.03      0.26      0.75      0.05      0.45      0.19       272
 Somewhat Shareable       0.34      0.23      0.82      0.27      0.43      0.17      2834

        avg / total       0.59      0.25      0.79      0.34      0.44      0.19      9911



In [17]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.17559933953156273, ' n_tokens_content'),
 (0.1711563924967669, ' avg_negative_polarity'),
 (0.16184117659009292, ' avg_positive_polarity'),
 (0.13482126895900384, ' rate_negative_words'),
 (0.13151342404179445, ' rate_positive_words'),
 (0.11529010038469024, ' n_tokens_title'),
 (0.10977829799608885, ' title_sentiment_polarity')]